# Quantification des incertitudes

## Résumé

Dans cette page, nous utilisons le cas de sollicitation d'une poutre en flexion dans le but d'identifier la loi qui s'ajuste à un échantillon unidimensionnel donné. Plus précisément, nous mettons en œuvre une démarche d'inférence statistique visant à identifier la distribution théorique sous-jacente.

[Pour voir ce Jupyter Notebook, nous conseillons NBViewer.](https://nbviewer.org/github/mbaudin47/otsupgalilee-eleve/blob/master/2-Quantification/Exercice-cantilever-beam-fit-distribution.ipynb)

# Introduction

On considère le modèle de [poutre encastrée](https://github.com/mbaudin47/otsupgalilee-eleve/blob/master/0-Deroulement/Poutre-encastree.md). Le module de Young de la poutre est incertain et varie entre 65 et 75 GPa. Ces valeurs correspondent à la fibre de verre, un matériau souvent utilisé pour la fabrication des plongeoirs.

Dans le cadre de cet exercice pédagogique, la distribution **réelle** (utilisée pour générer les données) est :

$$
\text{Beta}\left(\alpha = 0.9, \, \beta = 2.6, \, a = 65, \, b = 75\right)
$$

La loi précédente est la loi Beta généralisée, définie sur l'intervalle $[a,b]$ avec les paramètres de forme $\alpha$ et $\beta$.

Dans le cadre de cet exercice, on fait l'hypothèse que cette distribution est inconnue. On dispose d'un échantillon de 229 mesures de $E$ dans le fichier `sample_E.csv`. L'objectif de cet exercice est d'identifier la loi qui s'ajuste le mieux à cet échantillon.

## Notes sur le contexte mécanique

- Dans une version précédente de cet exercice, le module d'Young considéré était compris entre $28 \times 10^6$ et $48 \times 10^6$, c'est-à-dire entre 28 MPa et 48 MPa. Ce module d'élasticité correspond approximativement à du caoutchouc. Ce dernier n'est pas un matériau approprié pour un plongeoir.

- Dans un contexte mécanique, une contrainte peut être exprimée en pascals, qui est l'unité du Système international. Le problème de cette unité est qu'elle mène à des nombres très grands, de l'ordre de $10^9$ pour le module d'Young. Cela peut poser des problèmes pour analyser les résultats, ainsi que des difficultés pour les méthodes numériques, par exemple pour déterminer le pas de différenciation utilisé dans le contexte de l'approximation du gradient par une formule de différences finies. Pour ces raisons, il peut être pertinent d'utiliser une unité plus appropriée comme le gigapascal. Une autre méthode consisterait à standardiser les données, ce qui permettrait de réduire l'amplitude des observations de module d'Young. Puisque le changement d'unité permet de résoudre le problème, c'est l'option que nous choisissons ici. 

In [1]:
import openturns as ot
import os

## Génération de l'échantillon

Pour faciliter la suite des exercices, la cellule suivante définit les paramètres exacts de la distribution. En pratique ces paramètres sont inconnus, mais ils permettent de vérifier que nos estimations sont correctes.

In [2]:
# Définition des paramètres exacts de la loi Beta
alpha_exact = 0.9
beta_exact = 2.6
a_exact = 65.0  # GPa
b_exact = 75.0  # GPa

# Affectation dans un vecteur de dimension 4 (Point)
# L'ordre standard dans OpenTURNS pour la loi Beta est [alpha, beta, a, b]
exact_parameters = ot.Point([alpha_exact, beta_exact, a_exact, b_exact])

# Affichage du vecteur
print(f"Vecteur des paramètres exacts : {exact_parameters}")

Vecteur des paramètres exacts : [0.9,2.6,65,75]


En pratique, il peut arriver que seul le fichier de données contenant les observations soit disponible. Dans notre contexte pédagogique, ces données sont simulées à partir d'une loi bêta dont les paramètres sont connus. Dans la cellule suivante, si le fichier n'est pas dans le répertoire courant, alors nous générons les observations.

In [3]:
generate_sample = not os.path.isfile("sample_E.csv")
if generate_sample:
    # Si le fichier n'est pas dans le répertoire courant, génère-le.
    ot.RandomGenerator.SetSeed(0)
    beta_exact = ot.Beta()
    beta_exact.setParameter(exact_parameters)
    beta_exact.setDescription(["Young Modulus (GPa)"])
    sample_size = 229
    sample = beta_exact.getSample(sample_size)
    sample.exportToCSVFile("sample_E.csv")

## Lecture des données

In [4]:
data = ot.Sample.ImportFromCSVFile("sample_E.csv")
data.setDescription(["Young Modulus"])  # Optionnel
n = data.getSize()
print(f"Sample size={n}")

Sample size=229


In [5]:
data[0:10]

,Young Modulus
0,65.61071
1,70.46924
2,65.26221
3,68.88013
4,69.91335
5,70.6468
6,68.12899
7,65.36817
8,65.99786
9,65.09173


## Question 1 : statistiques descriptives

**Questions.**

- Calculer les principales caractéristiques de l'échantillon : moyenne, écart-type, médiane, minimum, maximum, intervalle inter-quartiles (c'est-à-dire à 25% et 75%).

**Indications méthodologiques :**  

Utiliser les méthodes `computeMean()`, `computeStandardDeviation()`, `computeQuantile()`, `getMin()` et `getMax()` de la classe `Sample`.

## Question 2 : histogramme empirique

**Questions.**
- Utiliser la classe `HistogramFactory` pour dessiner l'histogramme empirique de l'échantillon.

## Question 3 : ajuster une loi Beta

**Estimation des paramètres.**

Nous allons estimer les paramètres de la loi Beta (les paramètres de forme $\alpha$ et $\beta$, ainsi que les bornes $a$ et $b$) à partir d'un échantillon de taille finie ($n=229$). Par conséquent, ces paramètres ne doivent pas être considérés comme des valeurs déterministes exactes, mais comme des estimateurs eux-mêmes soumis à une incertitude d'échantillonnage. La bibliothèque OpenTURNS permet de caractériser cette incertitude en calculant la distribution de probabilité des paramètres estimés.

**Questions.**

- Utiliser la classe `BetaFactory` pour estimer les paramètres de la loi Beta s'ajustant aux données. 
- Dessiner la densité de probabilité de cette loi ajustée.
- Afficher les paramètres estimés.
- Utiliser la méthode `buildEstimator` de la classe `BetaFactory` afin d'obtenir la distribution jointe du vecteur des paramètres.
- À partir de cette distribution jointe, extraire les lois marginales associées à chaque paramètre : $a$, $b$, $\alpha$ et $\beta$.
- Calculer et afficher l'intervalle de confiance bilatéral à 95 % pour chacun de ces quatre paramètres.
- Interprétation : Comment la taille de l'échantillon influence-t-elle la largeur de ces intervalles ? Quel serait l'impact d'une incertitude élevée sur les paramètres lors d'une étude de propagation d'incertitudes ultérieure ?


## Question 4 : réaliser le test de Kolmogorov-Smirnov

**Questions.**

- Utiliser la classe `FittingTest.Lilliefors` pour réaliser le test de Lilliefors. Ce test constitue une adaptation du test de Kolmogorov-Smirnov tenant compte de l'incertitude liée à l'estimation des paramètres de la distribution à partir de l'échantillon (contrairement au test de Kolmogorov-Smirnov classique qui suppose des paramètres connus).
- Calculer la p-valeur.
- Doit-on rejeter la loi Beta ou non ?

## Question 5 : diagramme quantile-quantile

**Questions.**

- Utiliser la classe `VisualTest.DrawQQplot` pour dessiner le diagramme quantile-quantile. 
- L'ajustement est-il satisfaisant ?

## Question 6 : sélection de modèle par le critère BIC

On souhaite déterminer quelle loi s'ajuste le mieux au sens du critère BIC (c'est-à-dire menant à la plus petite valeur du critère BIC). 

**Questions.**

- Comparer les lois suivantes du point de vue du critère : uniforme, gaussienne, beta, log-normale. Quelle est la loi qui est la meilleure du point de vue du critère BIC, c'est-à-dire la loi la plus parcimonieuse ?
- La méthode `FittingTest.BestModelBIC` permet de renvoyer la distribution réalisant le meilleur critère BIC dans une liste de distributions candidates. Pour créer cette liste, on peut utiliser la méthode `GetContinuousUniVariateFactories`. Quelle distribution ressort de cette méthode ? Est-elle satisfaisante du point de vue du diagramme quantile-quantile ?

Veuillez noter que la méthode `ot.FittingTest.BIC` présente un comportement distinct suivant que l'argument d'entrée est une fabrique (`Factory`) ou une loi de probabilité (`Distribution`).

- Si une fabrique (`Factory`) est fournie, le nombre de paramètres estimés est alors égal au nombre de paramètres de la loi de probabilité.

- Si une distribution (`Distribution`) est fournie, le nombre de paramètres estimés doit alors être défini par l'utilisateur. Dans le cas contraire, la configuration par défaut fixe le nombre de paramètres estimés au nombre de paramètres de la distribution.

**Référence**

- https://github.com/openturns/openturns/issues/2198

## Question 7 : histogramme

**Questions.**

- Utiliser la classe `HistogramFactory` pour créer la distribution de type histogramme qui s'ajuste aux données. 
- Utiliser le diagramme quantile-quantile pour valider graphiquement cette distribution. 
- Quelles sont les limitations méthodologiques de l'approche par histogramme (sensibilité au nombre de classes, absence de lissage, discontinuités) ?
- L'histogramme constitue-t-il une distribution satisfaisante dans ce cas particulier ? Justifiez votre réponse.
- Dans quel type d'études industrielles ou académiques cette méthode serait-elle préférable aux ajustements paramétriques classiques ?

## Question 8 : méthode de lissage à noyau

**Questions.**

- Utiliser la classe `KernelSmoothing` pour utiliser une méthode à noyau pour estimer la distribution. 
- Utiliser le diagramme quantile-quantile pour valider le résultat graphiquement.
- Utiliser la méthode `setBoundaryCorrection()` pour activer la correction aux bords. Qu'observez-vous sur la qualité de l'ajustement dans les régions extrêmes de la distribution ?

**Remarque.**

La méthode `setBoundingOption()` permet de spécifier à quel bord la correction s'applique, c'est-à-dire à gauche, à droite ou aux deux bords.

## Question 9 : synthèse

On souhaite comparer les trois distributions suivantes :

- la loi Beta,
- la distribution issue de l'estimation par lissage à noyau,
- l'histogramme empirique.

**Questions.**

Comparer les distributions précédentes du point de vue de la densité de probabilité : 

- quelles différences observez-vous ?
- quelle semble être la loi la plus appropriée ?